In [ ]:
from dataset import TorinoAquaDataset
from models.Unet2 import Unet2
import os
import torch
import torch.nn as nn
from torchinfo import summary
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from utils import MaskL1Loss, DiceLoss, VGGLoss
import torchvision
from torch.utils.tensorboard import SummaryWriter

LOAD_CHECKPOINT = True

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Unet2(2).to(device)

dataset = TorinoAquaDataset(no_mask=-1, factor=(10, 15))
dataloader = DataLoader(dataset, 1, True, num_workers=1)

if LOAD_CHECKPOINT and os.path.exists('TrainCheckpoint/'+model.__class__.__name__+'CheckpointLast.pth'):
    checkpoint_file = torch.load('TrainCheckpoint/'+model.__class__.__name__+'CheckpointLast.pth')
    model.load_state_dict(checkpoint_file['model'])
    print('Checkpoint loaded!')

In [ ]:
with torch.inference_mode():
    for i, data in enumerate(dataloader):
        inputs = data['input'].to(device)
        label = data['label'].to(device)
        logits = model(inputs)
        grid = torchvision.utils.make_grid([inputs[0], label[0], logits[0]], 3)
        torchvision.utils.save_image(
                        grid,
                        f'TrainImg/Inference{i+1}.png'
                    )
        if i+1 == 10:
            break
    